# Spectrum Holding Summary Document

Code within notebook tests the shapefile processing to present as human-readable summaries of client number extracts.

## Unique Area Function

The most challenging part of generating a holding summary is being able to remove overlapping HCIS cells from each area description entry. Removing overlapping HCIS cells is necessary to produce a two dimensional, human readable dataset.

In [ ]:

def getUniqueAreas(gdf):
    """Given a geodataframe, return a list of unique HCIS descriptions

    Args:
        gdf (GeoDataFrame): Client data geodataframe

    Returns:
        uniqueAreas: List of HCIS descriptions (list)
    """
    gdf = clientData

    areaStrings = gdf['AREA_DESCRIPTION'].tolist()
    uniqueAreas = []

    for string in areaStrings:
        uniqueAreas.append(list(string.split(', ')))

    newAreas = []
    i = 0
    # Loop through every hcis for comparison
    for hcis1 in uniqueAreas:
        
        j = 0
        
        for hcis2 in uniqueAreas:
            
            # Ensure comparison of different HCIS cells
            if i != j:
                
                union = set(hcis1).intersection(set(hcis2))
                
                # If union is 0, there aren't any matches
                if len(union) == 0:
                    j = j + 1
                    continue
                
                # Duplicate HCIS
                elif set(hcis1) == union:
                    uniqueAreas.pop(j)
                    
                else:
                    newAreas.append(list(union))
                    uniqueAreas[i] = list(set(hcis1) - union)
                    uniqueAreas[j] = list(set(hcis2) - union)
                    
            j = j + 1
            
        i = i + 1

    uniqueAreas.append(newAreas)
    return uniqueAreas

uniqueAreas = getUniqueAreas(shp)
#print(uniqueAreas)

In [ ]:
# Loop testing

newAreas = []
i = 0
# Loop through every hcis for comparison
for hcis1 in uniqueAreas:
    
    j = 0
    
    for hcis2 in uniqueAreas:
        
        # Ensure comparison of different HCIS cells
        if i != j:
            
            union = set(hcis1).intersection(set(hcis2))
            
            # If union is 0, there aren't any matches
            if len(union) == 0:
                j = j + 1
                continue
            
            # Duplicate HCIS
            elif set(hcis1) == union:
                uniqueAreas.pop(j)
                
            else:
                newAreas.append(list(union))
                uniqueAreas[i] = list(set(hcis1) - union)
                uniqueAreas[j] = list(set(hcis2) - union)
                
        j = j + 1
        
    i = i + 1

uniqueAreas.append(newAreas)

uniqueAreas = getUniqueAreas(shp)
print(unique)

In [ ]:
# build gdf from unique areas
bands = set(specLicence['LICENCE_CATEGORY_NAME'])
bands

holdings = pd.DataFrame({
    "HCIS" : [],
    'Total BW' : [],
    '1800 MHz Band' : [],
    '2 GHz Band' : [],
    '2.3 GHz Band' : [],
    '2.5 GHz Band' : [],
    '2.5 GHz Mid Band Gap' : [],
    '20 GHz Defence Band' : [],
    '26 GHz Band' : [],
    '3.4 GHz Band' : [],
    '30 GHz Defence Band' : [],
    '700 MHz Band' : [],
    '800 MHz Band' : [],
    'AWL - FSS Only' : [],
    'AWL - Standard' : []
})
holdings

In [ ]:
# ------------------------------------------------------
# -------- Build holdings helper functions -------------
# ------------------------------------------------------

def intersection (HCIS, row):
    """Returns list of HCIS that is in both input HCIS list and input row.

    Args:
        HCIS (List): List of HCIS identifiers.
        row (Dataframe Row): Row from a client dataframe or shapefile.

    Returns:
        isin (List): List of HCIS identifiers in both inputs.
    """
    temp = set(row['AREA_DESCRIPTION'])
    isin = [value for value in HCIS if value in temp]
    return isin

def removeHCIS (AreaDescription, uaHCIS):
    newHCIS = []
    return newHCIS

def updateHoldings (row, holdings, uaHCIS):
    return

HCIS = uniqueAreas[-1]
print(HCIS)

In [ ]:
# ------------------------------------------------------
# -------- Build holdings primary function -------------
# ------------------------------------------------------

def buildHoldings (uniqueAreas, shp):
    """Given a list of unique area defined by HCIS sequences 
    and a geodataframe, generate a holdings summary.

    Args:
        uniqueAreas (List of Lists): List containing unique HCIS sequences
        shp (Geodataframe): Client Data geodataframe

    Returns:
        holdings: Holdings summary geodataframe
    """
    
    # Create empty dataset
    holdings = pd.DataFrame({
    "HCIS" : [],
    'Total BW' : [],
    '1800 MHz Band' : [],
    '2 GHz Band' : [],
    '2.3 GHz Band' : [],
    '2.5 GHz Band' : [],
    '2.5 GHz Mid Band Gap' : [],
    '20 GHz Defence Band' : [],
    '26 GHz Band' : [],
    '3.4 GHz Band' : [],
    '30 GHz Defence Band' : [],
    '700 MHz Band' : [],
    '800 MHz Band' : [],
    'AWL - FSS Only' : [],
    'AWL - Standard' : []
    })
    
    for uaHCIS in uniqueAreas:
        
        for index, row in shp.iterrows():
            
            isin = intersection(uaHCIS, row)
            
            # If uaHCIS is in entry[HCIS]
            if len(isin) != 0:
                
                shp.at[index]['LICENCE_AREA_DESCRIPTION'] = removeHCIS(shp.iloc[index]['LICENCE_AREA_LOCATION'], uaHCIS)
                
                updateHoldings = (row, holdings, uaHCIS)
            
            else:
                
                continue
            
    return holdings